# Основные переменные

In [ ]:
"""
Неликвид:
С - по АБС группам (А, В, С)
N-- по индексу неликвидности (N--, N-, N, N+, N++)

Основные переменные:

lst_top_20       - список ТОП 20
lst_brand_name   - список брендов ['SKF', 'FAG', 'NSK', 'TIMKEN', 'SNR', 'KOYO']

df_stock         - все остатки понедельно
df_stock_tip     - все остатки понедельно, сгруппированы по TIP/ГОСТ (group_by)

df_sales         - данные по всем продажам и счетам
df_order         - отгрузки
df_bill          - выставлены счета, без отгрузки
df_order_tip     - все продажи понедельно, сгруппированы по TIP/ГОСТ (pivot_table)

lst_stock_sort   - отсортированный список недель (остатки = продажи)
cg               - список для контроля данных по code_1c и GOST 

""";

# Загрузка библиотек

In [ ]:
# загрузка библиотек
import pandas as pd
# устанавливаем отображение до 3-х знаков после запятой
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import numpy as np
import re

import datetime as DT
from datetime import datetime, date

from xlsxwriter.workbook import Workbook

In [ ]:
# время начала выполнения проекта
time_project_start = DT.datetime.now(DT.timezone.utc).astimezone()

In [5]:
# функции для загрузки данных
import os
import sys

# определяем абсолютный путь к папке с функциями
abs_path = os.path.abspath('C:/Users/Andrew/08_energy/01_functions')
sys.path.append(abs_path)

# загружаем функции:
# загрузка данных по продажам
# загрузка данных по остаткам (2024 и 2025 год)
from def_load_data import load_of_sales, load_of_stock_by_week

# загружаем функцию сортировки недель
from def_weeks_sort import weeks_sort

# Журнал событий

In [ ]:
class PrinterWithStorage:
    # инициализация списка для хранения данных
    def __init__(self):
        self.cg = []
    
    def __call__(self, *args):
        print(*args)  # печать аргументов
        if args:      
            self.cg.append(args[0] if len(args) == 1 else args) # Добавление аргументов
    
    def show_storage(self):
        # метод для вывода содержимого cg
        print("Содержимое cg:", self.cg)

# printer(1, 2, 3)  # добавит кортеж (1, 2, 3) в cg

# Варианты вывода cg:
# print(printer.cg)              # прямой доступ
# printer.show_storage()         # через метод 
# print(printer)                 # через __repr__
# print(vars(printer)['cg')      # через словарь атрибутов

# использование:
printer = PrinterWithStorage()
printer("Журнал событий:")
printer("Время начала выполнения проекта:", str(time_project_start))
printer(" ")


Журнал событий:
Время начала выполнения проекта: 2025-07-08 15:40:18.203387+03:00
 


# stop_errors

In [7]:
# если объявить переменную, учтенные ошибки будут игнорироваться
# stop_errors ='ignore'

# Загрузка данных

## Загружаем продажи: df_sales

In [ ]:
# формируем словарь с типом данных
columns_str_sales = ['bill_date','bill_number','bill_close_date',
                     'bill_close_why','bill_manager','source',
                     'order_date','order_number','manager_client',
                     'manager_additional_1','manager_additional_2',
                     'manager_additional_3','client_inn','client_name','head_inn',
                     'head_name','code_1c','tip_product','name_1c','purch_inn',
                     'purch_name','purch_date','purch_number',
                     ]

columns_float_sales = ['bill_count','bill_price','order_count',
                       'order_weight_psc','order_weight','order_price',
                       'order_sum','purch_count','purch_price',
                       'purch_sum','price_last_pur','sum_last_pur',
                       'cost_price','cost_sum','margin_master'
                       ]


dict_sales = dict.fromkeys(columns_str_sales, str)
dict_sales.update(dict.fromkeys(columns_float_sales, float))

# загрузка данных: продажи
df_sales = pd.read_excel(
    io=r'C:\Users\Andrew\08_energy\00_in_data\df_sales.xlsx',
    engine='openpyxl',
    header=0,
    converters=dict_sales)

## Загружаем остатки: df_stock

In [ ]:
# формируем словарь с типом данных
columns_str_stock = ['tip_product','brand','code_1c',
                     '01.10.2021 0:00:00','04.10.2021 0:00:00','11.10.2021 0:00:00','18.10.2021 0:00:00','25.10.2021 0:00:00','01.11.2021 0:00:00','08.11.2021 0:00:00','15.11.2021 0:00:00','22.11.2021 0:00:00','29.11.2021 0:00:00','06.12.2021 0:00:00','13.12.2021 0:00:00','20.12.2021 0:00:00','27.12.2021 0:00:00','10.01.2022 0:00:00','17.01.2022 0:00:00','24.01.2022 0:00:00','31.01.2022 0:00:00','07.02.2022 0:00:00','14.02.2022 0:00:00','21.02.2022 0:00:00','28.02.2022 0:00:00','07.03.2022 0:00:00','14.03.2022 0:00:00','21.03.2022 0:00:00','28.03.2022 0:00:00','04.04.2022 0:00:00','11.04.2022 0:00:00','18.04.2022 0:00:00','25.04.2022 0:00:00','02.05.2022 0:00:00','09.05.2022 0:00:00','16.05.2022 0:00:00','23.05.2022 0:00:00','30.05.2022 0:00:00','06.06.2022 0:00:00','13.06.2022 0:00:00','20.06.2022 0:00:00','27.06.2022 0:00:00','04.07.2022 0:00:00','11.07.2022 0:00:00','18.07.2022 0:00:00','25.07.2022 0:00:00','01.08.2022 0:00:00','08.08.2022 0:00:00','15.08.2022 0:00:00','22.08.2022 0:00:00','29.08.2022 0:00:00','05.09.2022 0:00:00','12.09.2022 0:00:00','19.09.2022 0:00:00','26.09.2022 0:00:00','03.10.2022 0:00:00','10.10.2022 0:00:00','17.10.2022 0:00:00','24.10.2022 0:00:00','31.10.2022 0:00:00','07.11.2022 0:00:00','14.11.2022 0:00:00','21.11.2022 0:00:00','28.11.2022 0:00:00','05.12.2022 0:00:00','12.12.2022 0:00:00','19.12.2022 0:00:00','26.12.2022 0:00:00','09.01.2023 0:00:00','16.01.2023 0:00:00','23.01.2023 0:00:00','30.01.2023 0:00:00','06.02.2023 0:00:00','13.02.2023 0:00:00','20.02.2023 0:00:00','27.02.2023 0:00:00','06.03.2023 0:00:00','13.03.2023 0:00:00','20.03.2023 0:00:00','27.03.2023 0:00:00','03.04.2023 0:00:00','10.04.2023 0:00:00','17.04.2023 0:00:00','24.04.2023 0:00:00','01.05.2023 0:00:00','08.05.2023 0:00:00','15.05.2023 0:00:00','22.05.2023 0:00:00','29.05.2023 0:00:00','05.06.2023 0:00:00','12.06.2023 0:00:00','19.06.2023 0:00:00','26.06.2023 0:00:00','03.07.2023 0:00:00','10.07.2023 0:00:00','17.07.2023 0:00:00','24.07.2023 0:00:00','31.07.2023 0:00:00','07.08.2023 0:00:00','14.08.2023 0:00:00','21.08.2023 0:00:00','28.08.2023 0:00:00','04.09.2023 0:00:00','11.09.2023 0:00:00','18.09.2023 0:00:00','25.09.2023 0:00:00','02.10.2023 0:00:00','09.10.2023 0:00:00','16.10.2023 0:00:00','23.10.2023 0:00:00','30.10.2023 0:00:00','06.11.2023 0:00:00','13.11.2023 0:00:00','20.11.2023 0:00:00','27.11.2023 0:00:00','04.12.2023 0:00:00','11.12.2023 0:00:00','18.12.2023 0:00:00','25.12.2023 0:00:00','01.01.2024 0:00:00','08.01.2024 0:00:00','15.01.2024 0:00:00','22.01.2024 0:00:00','29.01.2024 0:00:00','05.02.2024 0:00:00','12.02.2024 0:00:00','19.02.2024 0:00:00','26.02.2024 0:00:00','04.03.2024 0:00:00','11.03.2024 0:00:00','18.03.2024 0:00:00','25.03.2024 0:00:00','01.04.2024 0:00:00','08.04.2024 0:00:00','15.04.2024 0:00:00','22.04.2024 0:00:00','29.04.2024 0:00:00','06.05.2024 0:00:00','13.05.2024 0:00:00','20.05.2024 0:00:00','27.05.2024 0:00:00','03.06.2024 0:00:00','10.06.2024 0:00:00','17.06.2024 0:00:00','24.06.2024 0:00:00','01.07.2024 0:00:00','08.07.2024 0:00:00','15.07.2024 0:00:00','22.07.2024 0:00:00','29.07.2024 0:00:00','05.08.2024 0:00:00','12.08.2024 0:00:00','19.08.2024 0:00:00','26.08.2024 0:00:00','02.09.2024 0:00:00','09.09.2024 0:00:00','16.09.2024 0:00:00','23.09.2024 0:00:00','30.09.2024 0:00:00','07.10.2024 0:00:00','14.10.2024 0:00:00','21.10.2024 0:00:00','28.10.2024 0:00:00','04.11.2024 0:00:00','11.11.2024 0:00:00','18.11.2024 0:00:00','25.11.2024 0:00:00','02.12.2024 0:00:00','09.12.2024 0:00:00','16.12.2024 0:00:00','23.12.2024 0:00:00','06.01.2025 0:00:00','13.01.2025 0:00:00','20.01.2025 0:00:00','27.01.2025 0:00:00','03.02.2025 0:00:00','10.02.2025 0:00:00'
                       ]

dict_stock = dict.fromkeys(columns_str_stock, str)

In [10]:
# загрузка данных: остатки
df_stock = pd.read_excel(
    io=r'C:\Users\Andrew\08_energy\00_in_data\df_stock.xlsx',
    engine='openpyxl',
    header=0,
    converters=dict_stock
    )

# Предобработка данных

## Информация по данным

In [11]:
printer('Информация по данным: продажи')
printer('df_sales[code_1c].unique():', len(df_sales['code_1c'].unique()) )
printer('df_sales[tip_product].unique():', len(df_sales['tip_product'].unique()) )
printer('df_sales.shape:', df_sales.shape )
printer(" ")

Информация по данным: продажи
df_sales[code_1c].unique(): 101
df_sales[tip_product].unique(): 11
df_sales.shape: (12368, 42)
 


In [12]:
printer('Информация по данным: остатки')
printer('df_stock[code_1c].unique():', len(df_stock['code_1c'].unique()) )
printer('df_stock[tip_product].unique():', len(df_stock['tip_product'].unique()) )
printer('df_stock.shape:', df_stock.shape )
printer(" ")

Информация по данным: остатки
df_stock[code_1c].unique(): 77
df_stock[tip_product].unique(): 11
df_stock.shape: (77, 177)
 


## Проверка дубликатов

In [13]:
printer('Проверка дубликатов:')
printer('Продажи df_sales.duplicated():', len(df_sales[df_sales.duplicated()]) )
printer('Остатки df_stock.duplicated():', len(df_stock[df_stock.duplicated()]) )
printer(" ")

Проверка дубликатов:
Продажи df_sales.duplicated(): 0
Остатки df_stock.duplicated(): 0
 


## df_sales

### Формируем order_uniq, bill_uniq

In [ ]:
df_sales['order_uniq'] = df_sales['order_number'] + df_sales['order_date']
df_sales['bill_uniq'] = df_sales['bill_number'] + df_sales['bill_date']

In [ ]:
printer('Обработка_продаж: df_sales')
printer('добавляем order_uniq, bill_uniq')

printer('df_sales[order_number]',  len(df_sales['order_number'].unique())  )
printer('df_sales[bill_number]',  len(df_sales['bill_number'].unique())  )
printer('df_sales[order_uniq]',  len(df_sales['order_uniq'].unique())  )
printer('df_sales[bill_uniq]',  len(df_sales['bill_uniq'].unique())  )
printer(" ")
printer('df_sales.shape',  df_sales.shape  )

### datetime

In [ ]:
df_sales[['bill_date', 'bill_close_date', 'order_date']].head(2)

In [ ]:
# устанавливаем формат datetime
date_columns_list = ['bill_date', 'bill_close_date', 'order_date']
df_sales[date_columns_list] = (
    df_sales[date_columns_list].apply(pd.to_datetime, format = '%Y-%m-%d')
)

In [ ]:
printer('ПРОДАЖИ:    начало и конец периода'  )
printer(" ")
printer('df_sales[bill_date, bill_close_date, order_date] -- замена_str_на_datetime' )
printer('df_sales.shape             -- ', df_sales.shape )
printer('df_sales[order_date].min() -- ', df_sales['order_date'].min() )
printer('df_sales[order_date].max() -- ', df_sales['order_date'].max() )
printer('df_sales  interval         -- ', df_sales['order_date'].max() - df_sales['order_date'].min() )
printer(" ")

### bill_sum

In [ ]:
df_sales['bill_sum'] = df_sales['bill_count'] * df_sales['bill_price']

### head_inn

In [ ]:
# заполняем head_inn
# client_inn, head_inn заполним нулевым значением, поменяем тип данных на int
df_sales[['client_inn', 'head_inn']] = df_sales[['client_inn', 'head_inn']].fillna(0)
df_sales['client_inn'] = df_sales['client_inn'].map(int)
df_sales['head_inn'] = df_sales['head_inn'].map(int)


# добавляем head_inn_uniq
# заполняем значение head_inn из head_inn, если head_inn равно нулю
def find_inn(row):
    #print(row.iloc[-1)
    head_inn = row.iloc[-1] if row.iloc[-1] !=0 else row.iloc[-2]
    return pd.Series([head_inn], index=['head_index'])

df_sales['head_inn_uniq'] = df_sales[['client_inn', 'head_inn']].apply(find_inn, axis=1)

In [ ]:
printer('df_sales -- заполняем head_inn')
printer('уникальные клиенты по client_inn, head_inn_uniq')
printer('len(df_sales[client_inn].unique()', len(df_sales['client_inn'].unique()) )
printer('len(df_sales[head_inn_uniq].unique()', len(df_sales['head_inn_uniq'].unique()) )
printer(" ")

## df_stock

In [ ]:
printer('Обработка_остатков: df_stock' )
printer('Удаляем из данных: [,000 \xa0]' )
printer('Заголовки: формируем номера недель [01.11.2022 0:00:00 -> 44-2022]' )
printer(" ")

### заменяем Nan '\xa0'

In [ ]:
# остатки, обработка данных
# заменяем Nan '\xa0' - пробел в числах, убираем ',000', меняем тип данных
# отсекаки три столбца в начале и tr в конце
for column in df_stock.columns.values[3:]:
    df_stock[column] = df_stock[column].str.replace(',000', '', regex=True)
    df_stock[column] = df_stock[column].str.replace('\xa0', '')
    df_stock[column] = df_stock[column].str.replace('2\xa0478', '')
    df_stock[column] = df_stock[column].fillna(0)
    df_stock[column] = df_stock[column].astype('float')

### Меняем дату на номер недели: 01.11.2022 0:00:00 -> 44-2022

In [ ]:
# остатки первая и последняя дата в периоде
# без сортировки, порядок формируется при выгрузке данных
stock_first_date = df_stock.columns.values[3]
stock_last_date = df_stock.columns.values[-1]

In [ ]:
# остатки, переименовывваем заголовки, получаем номер недели
# 01.11.2022 0:00:00 -> 44-2022 from datetime import datetime
list_headers = []

for index in df_stock.columns[3:]:
    app_data = datetime.strptime(index, '%d.%m.%Y %H:%M:%S').strftime('%V-%Y')
    list_headers.append(app_data)

list_headers.insert(0, df_stock.columns[2])
list_headers.insert(0, df_stock.columns[1])
list_headers.insert(0, df_stock.columns[0])

In [ ]:
printer('Проверка дубликатов в заголовках остатков:', len(set(list_headers)), len(list_headers))
printer(" ")

In [ ]:
if len(set(list_headers)) != len(list_headers):
    print(stop_errors)

In [ ]:
# меняем название заголовков
df_stock.columns = list_headers

In [ ]:
# остатки первая и последняя неделя в периоде
# без сортировки, порядок формируется при выгрузке данных
stock_first_week = df_stock.columns.values[3]
stock_last_week = df_stock.columns.values[-1]

In [ ]:
# заполняем нули
df_stock = df_stock.fillna(0)

In [ ]:
# меняем тип данных на строковый
df_stock[['code_1c', 'brand', 'tip_product',]] = df_stock[[
          'code_1c', 'brand', 'tip_product',]].astype(str)

In [ ]:
printer('df_stock -- заполняем нулями, меняем на str'  )
printer('df_stock.fillna, [code_1c, tip_product, iso, brand].astype(str)'  )
printer('df_stock[code_1c].unique() -- после fillna_str:', len(df_stock['code_1c'].unique()))
printer('df_stock[tip_product].unique()    -- после fillna_str:', len(df_stock['tip_product'].unique()))
printer('df_stock.shape             -- после fillna_str:', df_stock.shape)
printer(" ")

# Формируем df_order

In [ ]:
# формируем уникальные номера bill_uniq, order_uniq

# df_sales   - все данные (счета + отгрузки)
# df_order   - отгрузки
# df_bill    - счета без отгрузок

df_sales[['code_1c', 'order_number']] = df_sales[['code_1c', 'order_number']].fillna('-').astype(str)
#df_sales[['code_1c', 'order_number']] = df_sales[['code_1c', 'order_number']].astype(str)

df_sales['order_weight_psc'] = df_sales['order_weight_psc'].fillna(0).astype(float)
#df_sales['order_weight_psc'] = df_sales['order_weight_psc'].astype(float)

df_order = (df_sales[(df_sales['order_number'] != '-')]).copy()
df_bill  = (df_sales[(df_sales['order_number'] == '-')]).copy()

# Фильтруем: остатки есть, нет продаж по code_1c

In [ ]:
# получаем список code_1c по которым нет продаж, но есть остатки
lst_no_sales_c1c = list(set(df_stock['code_1c']) - set(df_order['code_1c']))

# получаем список code_1c по которым нет остатков, но есть продаж
lst_no_stock_c1c = list(set(df_order['code_1c']) - set(df_stock['code_1c']))

# получаем общий список
lst_no_sales_no_stock_c1c = lst_no_sales_c1c + lst_no_stock_c1c

print('df_stock.shape', df_stock.shape)
print('df_order.shape', df_order.shape)

# значения которые есть в списке помечаем False
df_stock['true_sales'] = df_stock['code_1c'].apply(lambda x: x in lst_no_sales_c1c)
df_order['true_stock'] = df_order['code_1c'].apply(lambda x: x in lst_no_stock_c1c)

# отфильтровываем
df_no_order = df_stock.loc[df_stock['true_sales'] == True ]
df_no_stock = df_order.loc[df_order['true_stock'] == True ]

print(len(lst_no_sales_c1c))
print(len(lst_no_stock_c1c))
print(len(lst_no_sales_no_stock_c1c))

print('df_no_order.shape', df_no_order.shape)
print('df_no_stock.shape', df_no_stock.shape)

# (min, max) даты (sales, stock) 

In [ ]:
# первая и последняя дата отгрузки (формат Timestamp)
sales_first_date = df_sales['order_date'].min()
sales_last_date = df_sales['order_date'].max()

In [ ]:
# первая и последняя неделя отгрузки (формат str)
# timestamp_string = "2023-07-21 15:30:45"
format_string = "%Y-%m-%d %H:%M:%S"
sales_first_week = datetime.strptime(
    str(df_sales['order_date'].apply(pd.to_datetime, format = '%d.%m.%Y').min()),
    format_string).strftime('%V-%Y')

In [ ]:
sales_last_week = datetime.strptime(
    str(df_sales['order_date'].apply(pd.to_datetime, format = '%d.%m.%Y').max()),
    format_string).strftime('%V-%Y')

# первая и последняя дата счета
bill_first_date = df_sales['bill_date'].min()
bill_last_date = df_sales['bill_date'].max()

#timestamp_string = "2023-07-21 15:30:45"
format_string = "%Y-%m-%d %H:%M:%S"
bill_first_week = datetime.strptime(
    str(df_sales['bill_date'].apply(pd.to_datetime, format = '%d.%m.%Y').min()),
    format_string).strftime('%V-%Y')

bill_last_week = datetime.strptime(
    str(df_sales['bill_date'].apply(pd.to_datetime, format = '%d.%m.%Y').max()),
    format_string).strftime('%V-%Y')

In [ ]:
printer(" ")
printer('Остатки          -- df_stock')
printer('Первая дата      -- stock_first_date:', stock_first_date  )
printer('Последняя дата   -- stock_last_date:', stock_last_date )
printer('Первая неделя    -- stock_first_week:', stock_first_week  )
printer('Последняя неделя -- stock_last_week:', stock_last_week  )
stock_first_datetime = datetime.strptime(stock_first_date, "%d.%m.%Y %S:%M:%H")
stock_last_datetime = datetime.strptime(stock_last_date, "%d.%m.%Y %S:%M:%H")
printer('Интервал         -- df_stock[columns]:', stock_last_datetime - stock_first_datetime  )
printer(" ")
printer('Продажи          -- df_sales[order_date]'  )
printer('Первая дата      -- sales_first_date:', sales_first_date  )
printer('Последняя дата   -- sales_last_date:', sales_last_date )
printer('Первая неделя    -- sales_first_week:', sales_first_week  )
printer('Последняя неделя -- sales_last_week:', sales_last_week  )
sales_first_datetime = datetime.strptime(str(sales_first_date), "%Y-%m-%d %S:%M:%H")
sales_last_datetime = datetime.strptime(str(sales_last_date), "%Y-%m-%d %S:%M:%H")
printer('Интервал         -- df_sales[order_date]:', sales_last_datetime - sales_first_datetime  )
printer(" ")
printer('Счета            -- df_sales[bill_date]'  )
printer('Первая дата      -- bill_first_date:', bill_first_date  )
printer('Последняя дата   -- bill_last_date:', bill_last_date )
printer('Первая неделя    -- bill_first_week:', bill_first_week  )
printer('Последняя неделя -- bill_last_week:', bill_last_week  )
printer('Интервал         -- df_sales[bill_date]:', bill_last_date - bill_first_date   )
printer(" ")

# АБС

## АБС объем продаж

In [ ]:
# заполняем нулем отсутсвующие значения в ГОСТ
df_order['tip_product'] = df_order['tip_product'].fillna('-').astype(str)

In [ ]:
printer(" ")
printer('Формируем продажи в шт по TIP')
printer('df_frec_tip = df_order.groupby([tip_product)'  )
printer('df_order[code_1c].unique()' , len(df_order['code_1c'].unique()) )
printer('df_order[tip_product].unique()' , len(df_order['tip_product'].unique()) )

In [ ]:
# формируем  продажи в шт по ГОСТ
df_frec_tip = (df_order.groupby(['tip_product'])
                   .agg({'order_uniq': 'count',
                         'order_count': 'sum',
                         'code_1c': 'nunique'})
                   .rename(columns = {'order_uniq':'order_freq',
                                      'order_count': 'order_psc',
                                      'code_1c': 'code_1c_count'})
                   .reset_index()
                   .sort_values(by='order_psc', ascending=False))

In [ ]:
printer(df_frec_tip.head(2))
printer(" ")
printer('df_frec_tip[tip_product].unique()' , len(df_frec_tip['tip_product'].unique()) )
printer('df_frec_tip[code_1c_count]', df_frec_tip['code_1c_count'].sum() )
printer('df_frec_tip.shape' , df_frec_tip.shape )
printer(" ")

In [ ]:
# формируем сумму продаж и процент продаж по каждому товару
# сумма всех продаж в рублях
sum_of_all_sales_rub = df_order['order_sum'].sum()

# формируем сумму продаж по TIP
df_abc_tip_sum = (df_order.groupby(['tip_product'])
                   .agg({'order_sum': 'sum'})
                   .rename(columns = { 'order_sum':'orders_sum_tip'})
                   .reset_index()
                   .sort_values(by='orders_sum_tip', ascending=False))

In [ ]:
# добавляем процент, сортировка sales_perc_rub
df_abc_tip_sum['sales_perc_rub'] = (
    df_abc_tip_sum['orders_sum_tip'] / sum_of_all_sales_rub) * 100
df_abc_tip_sum = df_abc_tip_sum.sort_values(by='sales_perc_rub', ascending=False)

# расчет кумулятивной суммы
df_abc_tip_sum['sales_cums_rub'] = df_abc_tip_sum['sales_perc_rub'].cumsum()

# добавляем АБС группы по объему продаж
df_abc_tip_sum['abc_tip'] = df_abc_tip_sum['sales_cums_rub'].apply(lambda x:
'A'   if x < 81 else
'B'   if x < 96 else
'C')

# заполняем все пропуски
df_abc_tip_sum = df_abc_tip_sum.fillna(0)

In [ ]:
printer('Формируем АБС по TIP (сумма продаж и процент продаж)')
printer('Сумма продаж в периоде:', sum_of_all_sales_rub)
printer('df_abc_tip_sum = df_order.groupby([tip_product)'  )
printer(" ")
printer('Объединяем АБС и частоту отгрузок (merge, df_abc_tip_sum, df_frec_tip, left)'  )
printer('df_abc_tip_sum[tip_product].unique() -- до:', len(df_abc_tip_sum['tip_product'].unique())  )
printer('df_abc_tip_sum.shape          -- до:', df_abc_tip_sum.shape  )
printer(" ")

# объединяем АБС + частота отгрузок
df_abc_tip_sum = pd.merge(df_abc_tip_sum, df_frec_tip, how='left', on=['tip_product', 'tip_product'])

printer('df_abc_tip_sum[tip_product].unique() -- после:', len(df_abc_tip_sum['tip_product'].unique())  )
printer('df_abc_tip_sum.shape          -- после:', df_abc_tip_sum.shape  )

In [ ]:
printer('Проверка рассчитанного процента АВС', df_abc_tip_sum['sales_perc_rub'].sum() )
printer('Проверка сортировки df_abc_tip_sum (верх и низ) и кумулятивного процента'  )
printer(" ")
printer(df_abc_tip_sum[['tip_product', 'sales_perc_rub', 'sales_cums_rub', 'abc_tip']].head(3))
printer(" ")
printer(df_abc_tip_sum[['tip_product', 'sales_perc_rub', 'sales_cums_rub', 'abc_tip']].tail(3))
printer(" ")

## АБС маржинальность

In [ ]:
# формируем маржинальности по TIP
df_abc_tip_marg = (df_order.groupby(['tip_product'])
                   .agg({'margin_master': ['sum', 'mean'],
                         'purch_price': ['sum', 'mean'],
                         'order_price': 'mean'})
                   .reset_index())
                   #.sort_values(by='margin_master', ascending=False))
# rename columns 
df_abc_tip_marg.columns = ['tip_product', 'margin_sum', 'margin_mean',
                            'purch_sum', 'purch_mean', 'price_mean']

In [ ]:
# добавляем процент по маржинальности, сортировка margin_sales_rub
sum_margn_rub = df_abc_tip_marg['margin_sum'].sum()

df_abc_tip_marg['margin_perc_rub'] = (df_abc_tip_marg['margin_sum'] / sum_margn_rub) * 100
df_abc_tip_marg = df_abc_tip_marg.sort_values(by='margin_perc_rub', ascending=False)

# расчет кумулятивной суммы
df_abc_tip_marg['margin_cums_rub'] = df_abc_tip_marg['margin_perc_rub'].cumsum()

# добавляем АБС группы для маржи
df_abc_tip_marg['abc_marg'] = df_abc_tip_marg['margin_cums_rub'].apply(lambda x:
'A'   if x < 81 else
'B'   if x < 96 else
'C')

In [ ]:
printer('Формируем АБС_маржинальность по ГОСТ, df_abc_gost_marg')
printer('Проверка процента АБС_маржинальность', df_abc_tip_marg['margin_perc_rub'].sum() )
printer('Сумма маржи в периоде', sum_margn_rub )
printer(" ")
printer('df_abc_tip_marg[tip_product].unique() -- до:' , len(df_abc_tip_marg['tip_product'].unique()) )
printer('df_abc_tip_marg.shape          -- до:' , df_abc_tip_marg.shape  )
printer(" ")

In [ ]:
# добавляем маржинальность к АБС df_abc_gost_sum + df_abc_gost_marg
df_abc_tip_sum = pd.merge(df_abc_tip_sum, df_abc_tip_marg, how='left',
                          on=['tip_product', 'tip_product'])

## АБС рентабельность

In [ ]:
# рассчитываем рентабельность по ГОСТ (маржа / оборот * 100%)
df_abc_tip_sum['rent_rub'] = (df_abc_tip_sum['margin_sum'] / 
                              df_abc_tip_sum['orders_sum_tip']) * 100

# добавляем АБС для рентабельности
df_abc_tip_sum['abc_rent'] = df_abc_tip_sum['rent_rub'].apply(lambda x:
'А+'  if x > 35 else
'A'   if x > 25 else
'B'   if x > 5 else
'C')

# сортировка по margin_perc_rub
df_abc_tip_sum = df_abc_tip_sum.sort_values(by='margin_sum', ascending=False)

In [ ]:
printer('Добавляем маржинальность к АБС (merge, df_abc_gost_sum, df_abc_gost_marg, left)')
printer('Рассчитываем рентабельность по ГОСТ (маржа / оборот * 100%)')
printer('df_abc_gost_sum[gost].unique() -- после' , len(df_abc_tip_sum['tip_product'].unique()) )
printer('df_abc_gost_sum.shape          -- после' , df_abc_tip_sum.shape  )
printer('Проверка сортировки df_abc_tip_marg (верх и низ) и кумулятивного процента'  )
printer('Есть маржа меньше нуля => есть больше 100%'  )
printer(" ")
printer(df_abc_tip_sum[['margin_sum', 'margin_perc_rub',
                            'margin_cums_rub', 'abc_marg', 'abc_rent']].head(3))
printer(" ")
printer(df_abc_tip_sum[['margin_sum', 'margin_perc_rub',
                            'margin_cums_rub', 'abc_marg', 'abc_rent']].tail(3))

## LMHO по весу

In [ ]:
df_order['order_weight_psc'].info()

In [ ]:
# расчитываем округленный вес для всех заказов
# округление данных до целого 0.01 -> 1.00
df_order['weight_psc_round'] = df_order['order_weight_psc'].apply(np.ceil)

# формируем данные ГОСТ и усредненный вес
df_tip_weight = (df_order.groupby(['tip_product'])
                   .agg({'weight_psc_round': 'mean'})
                   .sort_values(by='weight_psc_round', ascending=False)
                   .reset_index())

# добавляем LMHO для веса
df_tip_weight['lmho'] = df_tip_weight['weight_psc_round'].apply(lambda x:
'H'  if x > 30 else
'M'  if x > 3 else
'L'  if x > 0 else
'O')

In [ ]:
# добавляем LMHO
df_abc_tip_sum = pd.merge(df_abc_tip_sum, df_tip_weight,
                          how='left', on=['tip_product', 'tip_product'])

In [ ]:
printer('Добавляем LMHO_по_весу (merge, df_abc_tip_sum, df_tip_weight, left)')
printer('df_tip_weight[tip_product].unique()' , len(df_tip_weight['tip_product'].unique()) )
printer('df_tip_weight.shape' , df_tip_weight.shape  )
printer('df_abc_tip_sum[tip_product].unique() -- после' , len(df_abc_tip_sum['tip_product'].unique()) )
printer('df_abc_tip_sum.shape                  --после' , df_abc_tip_sum.shape  )
printer(" ")

In [ ]:
lst_names_of_group = ["L", "M", "H", "O"]
lst_df_group = []

In [ ]:
for group in lst_names_of_group:
    # фильтруем по группе
    df_weight = (df_abc_tip_sum.loc[df_abc_tip_sum['lmho']==group])
    sum_sales_weight = df_weight['orders_sum_tip'].sum()
    # добавляем процент, сортировка sales_perc_rub
    df_weight['perc_rub_'+group] = (df_weight['orders_sum_tip']
                                    / sum_sales_weight) * 100
    df_weight = df_weight.sort_values(by='perc_rub_'+group, ascending=False)
    # расчет кумулятивной суммы
    df_weight['cumsum_'+group] = df_weight['perc_rub_'+group].cumsum()
    # добавляем АБС группы по объему продаж для каждой группы
    df_weight['abc_'+group] = df_weight['cumsum_'+group].apply(lambda x:
    'A'+group   if x < 81 else
    'B'+group   if x < 96 else
    'C'+group )
    
    # оставляем только TIP и группу
    df_weight = df_weight[['tip_product', 'abc_'+group]]
    
    # добавляем LMHO_по_группе
    df_abc_tip_sum = pd.merge(df_abc_tip_sum, df_weight, how='left',
                              on=['tip_product', 'tip_product'])

In [ ]:
# заполняем пустые значения
lst_adc_weight = ['abc_L', 'abc_M', 'abc_H', 'abc_O']
for index in lst_adc_weight:
    df_abc_tip_sum[index] = df_abc_tip_sum[index].fillna('-')

# добавляем столбец в который помещаем значения четырех групп
df_abc_tip_sum['adc_w'] = (df_abc_tip_sum['abc_L']+df_abc_tip_sum['abc_M']
                            +df_abc_tip_sum['abc_H']+df_abc_tip_sum['abc_O'])
# убираем тире
df_abc_tip_sum['adc_w'] = df_abc_tip_sum['adc_w'].str.replace('-', '')

In [ ]:
printer('Добавляем LMHO_по_группе (merge, df_abc_tip_sum, df_weight, left)')
printer('df_weight[tip_product].unique()' , len(df_weight['tip_product'].unique()) )
printer('df_weight.shape' , df_weight.shape  )
printer('df_abc_tip_sum[tip_product].unique() -- после' , len(df_abc_tip_sum['tip_product'].unique()) )
printer('df_abc_tip_sum.shape                  --после' , df_abc_tip_sum.shape  )
printer(" ")

# Понедельные остатки по TIP

In [ ]:
# формируем список недель (неупорядочен)
lst_stock_columns_weeks = list(
    # срез по номерам столбцов
    df_stock.columns.values[
                            df_stock.columns.get_loc('code_1c')+1:
                            df_stock.columns.get_loc('true_sales')
                            ]
                               )

In [ ]:
# создаём словарь для агрегации: 
# {колонка: функция} для недель и уникальные значения для остального
dict_stock_columns_weeks = {col: 'sum' for col in lst_stock_columns_weeks}

# добавляем code_1c and brand
dict_stock_columns_weeks.update({
    'code_1c': 'unique',
    'brand': 'unique'
})

In [ ]:
# groupby
df_stock_tip = (df_stock.groupby(['tip_product'])
                               .agg(dict_stock_columns_weeks)
                               .reset_index()
                               .sort_values(by='tip_product', ascending=False)
                    )
df_stock_tip.head(2)

# Понедельные продажи по TIP

In [ ]:
# заменяем дату отгрузки на номер недели
from datetime import datetime

# order_date  2022-08-22 00:00:00 -> 01-2022
df_order['order_date'] = df_order['order_date'].apply(pd.to_datetime, format = '%d.%m.%Y')
df_order['order_date'] = df_order['order_date'].astype(str)
df_order['order_week'] = df_order['order_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%V-%Y'))

In [ ]:
# перед формирование меняем тип данных, иначе появляются дубли
df_order['tip_product'] = df_order['tip_product'].fillna('-').astype(str)

In [ ]:
# формируем продажи в шт понедельно
df_order_tip = df_order.pivot_table(
                                      index=['tip_product'],
                                      columns='order_week',
                                      values='order_count',
                                      aggfunc='sum', fill_value=0).reset_index(drop=False)

In [ ]:
# формируем продажи gost + code_1c
df_order_tip_h = df_order.pivot_table(
                                      index=['tip_product'],
                                      values='code_1c',
                                      aggfunc='sum', fill_value=0).reset_index(drop=False)

In [ ]:
# получаем список недель, удаляем столбец ГОСТ
list_weeks_for_sort = list(df_order_tip.columns.values[1:])

# задаем тип данных
for index in list_weeks_for_sort:
    df_order_tip[index] = df_order_tip[index].astype(float)

In [ ]:
# разделяем код 1с в ГОСТ добавляем \n
df_order_tip_h['code_1c'] = df_order_tip_h['code_1c'].str.replace('00-', '\n00-')
df_order_tip_h['code_1c'] = df_order_tip_h['code_1c'].str.replace('П00', '\nП00')
# убираем \n в начале code_1c, -> разрезаем, создаем списки
df_order_tip_h['code_1c'] = df_order_tip_h['code_1c'].apply(lambda x: re.sub("^\n", "", x))
df_order_tip_h['code_1c'] = df_order_tip_h['code_1c'].apply(lambda x: x.splitlines(False))

In [ ]:
# убираем дубли code_1c которые образуются из-за сложения по неделям
df_order_tip_h['code_1c'] = df_order_tip_h['code_1c'].apply(lambda x: list(set(x)))

In [ ]:
printer('Продажи, формируем номер недели в заголовке (order_date),  2022-08-22 00:00:00 -> 01-2022')
printer('Понедельные_продажи_по_TIP: df_order_tip = df_order.pivot_table([tip_product)')
printer('len(df_order_tip[tip_product)', len(df_order_tip['tip_product'])  )
printer('df_order_tip.shape', df_order_tip.shape  )
printer(" ")        

In [ ]:
# в продажи по госту добавляем code_1c
df_order_tip = pd.merge(df_order_tip, df_order_tip_h, how='left', on=['tip_product', 'tip_product'])

printer('В продажи по госту добавляем code_1c (merge, df_order_tip, df_order_tip_h, left)')
printer('df_order_gost[gost].unique() -- после' , len(df_order_tip['tip_product'].unique()) )
printer('df_order_gost.shape          -- после' , df_order_tip.shape  )
printer(" ")

# Приравниваем количество недель

In [ ]:
printer(" ")
printer('Приравниваем_количество_недель (df_order_tip, df_stock_tip)')
printer('df_order_gost.shape  -- до' , df_order_tip.shape  )
printer('df_stock_gost.shape  -- до' , df_stock_tip.shape  )

In [ ]:
# формируем разницу в периоде df_stock_gost, df_order_gost
month_drop_from_stock = list( (set(set(df_stock_tip.columns.values) - set(df_order_tip.columns.values))) -
                       set(['brand'])
                       )

# формируем разницу в периоде df_order_gost, df_stock_gost
month_drop_from_order = list(set(df_order_tip.columns.values) - set(df_stock_tip.columns.values))

In [ ]:
print('Столбцы - остатки минус продажи:', month_drop_from_stock)
print('Столбцы - продажи минус остатки:', month_drop_from_order)

In [ ]:
# удаляем несоответсвие 
df_stock_tip = df_stock_tip.drop(month_drop_from_stock, axis = 1)
df_order_tip = df_order_tip.drop(month_drop_from_order, axis = 1)

In [ ]:
# проверяем разницу в периоде. В остатках остается ['brand']
print(list(set(df_stock_tip.columns.values) - set(df_order_tip.columns.values)))
print(list(set(df_order_tip.columns.values) - set(df_stock_tip.columns.values)))

In [ ]:
# получаем список недель без ГОСТ (в начале) и code_1c (в конце)
list_weeks_for_sort = list(df_order_tip.columns.values[1:-1])

In [ ]:
# сортируем список
lst_stock_sort = weeks_sort(list_weeks_for_sort)

In [ ]:
# формируем порядок столбцов для остатков и продаж
df_stock_tip = df_stock_tip[lst_stock_sort+['tip_product', 'code_1c', 'brand', ]]
df_order_tip = df_order_tip[lst_stock_sort+['tip_product', 'code_1c',]]

In [ ]:
# меняем тип данных на str
df_stock_tip[['tip_product', 'brand']] = df_stock_tip[['tip_product', 'brand']].astype(str)
df_order_tip['tip_product'] = df_order_tip['tip_product'].astype(str)

In [ ]:
printer('Столбцы - продажи минус остатки:', month_drop_from_order  )
printer('Столбцы - остатки минус продажи:', month_drop_from_stock  )
printer('df_order_tip.shape  -- после' , df_order_tip.shape  )
printer('df_stock_tip.shape  -- после' , df_stock_tip.shape  )
printer(" ")
printer('Проверяем разницу в периоде. В остатках остается [brand]')
printer('Столбцы - остатки минус продажи:',
    list(set(df_stock_tip.columns.values) - set(df_order_tip.columns.values))
)
printer('Столбцы - продажи минус остатки:',
    list(set(df_order_tip.columns.values) - set(df_stock_tip.columns.values))
)

# Преобразуем list(code_1c) в str(code_1c)

In [ ]:
df_stock_tip['code_1c'] = df_stock_tip['code_1c'].apply(lambda x: ', '.join(map(str, x)))

In [ ]:
df_order_tip['code_1c'] = df_order_tip['code_1c'].apply(lambda x: ', '.join(map(str, x)))

# Приравниваем остатки и продажи

In [ ]:
printer(" ")
printer('Приравниваем_остатки_и_продажи')
printer('Остатки -- len(df_stock_tip[tip].unique():', len(df_stock_tip['tip_product'].unique()) )
printer('Остатки -- df_stock_tip.shape', df_stock_tip.shape )
printer('Продажи -- len(df_order_tip[tip].unique():', len(df_order_tip['tip_product'].unique()) )
printer('Продажи -- df_order_tip.shape', df_order_tip.shape )

In [ ]:
# получаем список ГОСТ по которым нет продаж, но есть остатки
lst_no_sales = (list(set(df_stock_tip['tip_product'])
                     - set(df_order_tip['tip_product']))
               )
lst_no_stock = (list(set(df_order_tip['tip_product'])
                     - set(df_stock_tip['tip_product']))
               )
lst_no_sales_no_stock = lst_no_sales+lst_no_stock

printer('Длинна lst_no_sales TIP (нет продаж, но есть остатки):', len(lst_no_sales) )
printer('Длинна lst_no_stock TIP (нет остатков, но есть продажи):', len(lst_no_stock) )
printer('Длинна lst_no_sales_stock общего списка:', len(lst_no_sales_no_stock) )

In [ ]:
# значения которые есть в списке помечаем False
df_stock_tip['true_sales'] = df_stock_tip['tip_product'].apply(lambda x: x in lst_no_sales_no_stock)
df_order_tip['true_stock'] = df_order_tip['tip_product'].apply(lambda x: x in lst_no_sales_no_stock)

# отфильтровываем
df_no_order = df_stock_tip.loc[df_stock_tip['true_sales'] == True ]
df_no_stock = df_order_tip.loc[df_order_tip['true_stock'] == True ]
df_stock_tip = df_stock_tip.loc[df_stock_tip['true_sales'] == False ]
df_order_tip = df_order_tip.loc[df_order_tip['true_stock'] == False ]

In [ ]:
printer('Формируем таблицы')
printer('df_no_order -- нет продаж но есть остатки (TIP):', len(df_no_order)  )
printer('df_no_stock -- нет остатков но есть продажи (TIP):', len(df_no_stock)  )
printer('df_no_order -- нет продаж но есть остатки (Code_1c):', len(set(df_no_order['code_1c'])) )
printer('df_no_stock -- нет остатков но есть продажи (Code_1c):', len(set(df_no_stock['code_1c'])) )
printer(" ")        

In [ ]:
code_1c_no_order = set(df_stock_tip['code_1c']) - set(df_order_tip['code_1c'])

code_1c_no_stock = set(df_order_tip['code_1c']) - set(df_stock_tip['code_1c'])
printer(" ")
printer('В данных остаются позиции по Code_1c, по которым нет остатков или нет продаж')
printer('code_1c_no_order -- нет продаж но есть остатки:', len(code_1c_no_order)  )
printer('code_1c_no_stock -- нет остатков но есть продажи:', len(code_1c_no_stock)  )

In [ ]:
printer('Удаляем столбцы: true_sales, true_stock')
printer(" ")

In [ ]:
# удаляем один столбец: true_sales, true_stock
df_stock_tip = df_stock_tip.drop('true_sales', axis=1)
df_order_tip = df_order_tip.drop('true_stock', axis=1)

In [ ]:
printer(" ")
printer('Длинна таблиц')
printer('df_order_tip[tip]:', len(df_order_tip['tip_product'])  )
printer('df_order_tip[tip].unique:', len(df_order_tip['tip_product'].unique())  )
printer('df_order_tip.shape:', df_order_tip.shape  )

printer('df_stock_tip[tip]:', len(df_stock_tip['tip_product'])  )
printer('df_stock_tip[tip].unique:', len(df_stock_tip['tip_product'].unique())  )
printer('df_stock_tip.shape:', df_stock_tip.shape  )

printer(" ")
printer('Проверяем разницу в заголовках. В остатках остается [brand]' )
printer( list(set(df_stock_tip.columns.values) - set(df_order_tip.columns.values)) )
printer( list(set(df_order_tip.columns.values) - set(df_stock_tip.columns.values)) )

In [ ]:
# только недели продаж, без ГОСТ и код_1с
lst_order_tip = df_order_tip.columns.values[:-2]

# Работа с остатками

In [ ]:
printer('Работа_с_остатками df_stock_tip:')
printer(" ")
printer( '- помечаем новые приходы (остаток на минус 5 неделю)')
printer( '- добавляем десять процентов по продажам за пол года')
printer( '- средние продажи и сколько распродавать склад')
printer( '- оборачиваемость склада (продажи / средний остаток в периоде)')
printer( '- добавляем стоимость склада (цена продажи и цена закупки)')
printer( '- добавляем xyz')
printer(" ")        
printer('[mean_st_4] --  средний остаток за последние 5 недель'  )
printer('[true_receipt] -- True,  склад на 09-2024 неделю увеличенный на 20%'  )
printer('[sales_half_year] -- сумма продаж за последние пол года'  )
printer('[mean_sales_whole_period] -- средние продажи за весь период'  )
printer('[how_long_to_sale] -- сколько еще распродавать склада'  )
printer('[all_sales_whole_period] -- сумма всех продаж в периоде'  )
printer('[mean_stock_whole_period] -- средний остаток в периоде'  )
printer('[turnover] -- оборачиваемость: все продажи / средний остаток'  )
printer('lst_order_gost -- только недели продаж, без ГОСТ и код_1с'  )
printer(" ")
printer('df_price_mean -- средние цены закупки и продажи'  )
printer('[stock_sales1, stock_purch1] -- цена склада, начало периода'  )
printer('[stock_sales2, stock_purch2] -- цена склада, конец периода'  )
printer(" ")

## Помечаем неделю с новыми приходами

In [ ]:
# 08-2024
df_stock_tip.columns.values[-54]

In [ ]:
# помечаем новые приходы
# получаем средний остаток за последние 5 недель
df_stock_tip['mean_st_4'] = df_stock_tip[df_stock_tip.columns.values[-10:-5]].mean(axis=1)

In [ ]:
# помечаем True значения которые больше чем склад на 08-2024 неделю увеличенный на 20%
# row.iloc[-20] == '09-2024'
# row.iloc[-1] == 'mean_st_4'
def find_receipt1(row):
    head = 'нет' if row.iloc[-1] <= row.loc['08-2024']*1.2 else '09_неделя'
    return pd.Series([head], index=['head_index'])

df_stock_tip[['true_receipt']] = df_stock_tip.apply(find_receipt1, axis=1)

In [ ]:
# добавляем десять процентов по продажам за пол года
# создаем список недель за пол года
list_weeks_half_year = list(df_order_tip.columns.values[-37:-3])

In [ ]:
# сумма продаж за пол года
df_stock_tip['sales_half_year'] = df_order_tip[list_weeks_half_year].sum(axis=1)

# рассчитываем отношение продаж за пол года к
# среднему складу за 4 недели
df_stock_tip['percent'] = ((df_stock_tip['sales_half_year']/
                             df_stock_tip['mean_st_4'])*100).round(2)

In [ ]:
# получаем позиции по которым продажи менее 10%
# чем склад за последние 4 недели
df_stock_tip['true_percent'] = df_stock_tip['percent'].apply(lambda x:
                                                               'меньше_10' if x <= 10
                                                               else 'больше_10')

In [ ]:
printer(' Cредние продажи и сколько распродавать склад')

In [ ]:
# среднее значение продаж в неделю за весь период
df_stock_tip['mean_sales_whole_period'] = (df_order_tip[lst_stock_sort].mean(axis=1)).round(1)
# делим конечный остаток на средние продажи в неделю
df_stock_tip['how_long_to_sale'] = ((df_stock_tip[lst_stock_sort[-1]] /
                                    df_stock_tip['mean_sales_whole_period']
                             )).round(0)

In [ ]:
printer(' оборачиваемость склада = продажи / средний остаток в периоде ')

In [ ]:
# сумма всех продаж в периоде
df_stock_tip['all_sales_whole_period'] = (df_order_tip[lst_stock_sort].sum(axis=1)).round(0)

# средний остаток в периоде
df_stock_tip['mean_stock_whole_period'] = (df_order_tip[lst_stock_sort].mean(axis=1)).round(1)

# оборачиваемость: все продажи / средний остаток
df_stock_tip['turnover'] = (df_stock_tip['all_sales_whole_period']
                             / df_stock_tip['mean_stock_whole_period']).round(0)

In [ ]:
# добавляем стоимость склада (цена продажи и цена закупки)

In [ ]:
# для расчета стоимости склада в ценах продажи и ценах закупки
# df_price_mean - средние цены закупки и продажи
df_price_mean = (df_order.groupby(['tip_product'])
                   .agg({'purch_price': 'mean', 'order_price': 'mean'})
                   .rename(columns = {'purch_price':'price_purch_mean',
                                      'order_price':'price_sales_mean'})
                   .round(1)
                   .reset_index()
                   .sort_values(by='tip_product', ascending=True)
                   )

In [ ]:
lst_mean_price_drop1 = list(set(df_price_mean['tip_product']) - set(df_stock_tip['tip_product']))
lst_mean_price_drop2 = list(set(df_stock_tip['tip_product']) - set(df_price_mean['tip_product']))
print('Средние цены минус остатки (TIP):', lst_mean_price_drop1 )
print('Остатки минус средние цены (TIP):', lst_mean_price_drop2 )

In [ ]:
# получаем список индексов для удаления
lst_drop_index = df_price_mean[df_price_mean['tip_product'].isin(lst_mean_price_drop1)].index.tolist()

In [ ]:
# удаляем строки по индексу
df_price_mean = df_price_mean.drop(index=lst_drop_index).reset_index()

In [ ]:
# выравниваем позиции, делаем сортировку
df_price_mean = df_price_mean.sort_values(by='tip_product', ascending=False).reset_index(drop=True)
df_stock_tip = df_stock_tip.sort_values(by='tip_product', ascending=False).reset_index(drop=True)

In [ ]:
# добавляем стоимость склада (цена продажи и цена закупки)
# начало периода
df_stock_tip['stock_sales1'] = df_price_mean['price_sales_mean']*df_stock_tip[lst_stock_sort[1]]
df_stock_tip['stock_purch1'] = df_price_mean['price_purch_mean']*df_stock_tip[lst_stock_sort[1]]

In [ ]:
# добавляем стоимость склада (цена продажи и цена закупки)
# конец периода
df_stock_tip['stock_sales2'] = df_price_mean['price_sales_mean']*df_stock_tip[lst_stock_sort[-1]]
df_stock_tip['stock_purch2'] = df_price_mean['price_purch_mean']*df_stock_tip[lst_stock_sort[-1]]

In [ ]:
# добавляем данные на конец периода
df_stock_tip['last_stock'] = df_stock_tip[lst_stock_sort[-1]]

# Расcчитываем xyz для продаж

In [ ]:
# столбцы с номером недель
# lst_order_gost

def find_variab(row):
    # расчет по всем данным
    #f_variab = (np.std(row[1:)/np.mean(row[1:)*100).round(2)            
    f_variab = round( (np.std(row[row!=0])/np.mean(row[row!=0]))*100, 2)
    return pd.Series([f_variab], index=['variab'])

df_order_tip['variab'] = df_order_tip[lst_order_tip].apply(find_variab, axis=1)

In [ ]:
# добавляем XYZ для групп товара
df_order_tip['xyz'] = df_order_tip['variab'].apply(lambda x:
'Z'  if x > 150 else
'Y'  if x > 50 else
'O'  if x == 0 else
'X')

# Объединяем остатки АБС LMHO

In [ ]:
# задаем тип данных для ГОСТ перед объединением
df_stock_tip['tip_product'] = df_stock_tip['tip_product'].astype(str)
df_abc_tip_sum['tip_product'] = df_abc_tip_sum['tip_product'].astype(str)

In [ ]:
printer(" ")
printer('Объединяем_остатки_АБС_LMHO' )
printer('df_stock_tip = pd.merge(df_stock_tip, df_abc_tip_sum, on=[tip_product, tip_product], how=left)')
printer('Проверка размеров' )
printer('ABC -- df_abc_tip_sum.shape', df_abc_tip_sum.shape )
printer('ABC_gost -- len(df_abc_tip_sum[tip_product].unique()', len(df_abc_tip_sum['tip_product'].unique()) )
printer('Отгрузки_гост -- df_order_tip.shape', df_order_tip.shape )
printer('Отгрузки_гост -- len(df_order_tip[tip_product].unique())', len(df_order_tip['tip_product'].unique()) )

In [ ]:
printer(" ")
printer('Остатки_tip до -- df_stock_tip.shape:', df_stock_tip.shape )
printer('Остатки_tip до -- len(df_stock_tip[tip_product].unique()):', len(df_stock_tip['tip_product'].unique()) )
printer(" ")
printer('Продажи минус остатки (TIP):', list(set(df_abc_tip_sum['tip_product'])
                                                 - set(df_stock_tip['tip_product'])) )
printer('Остатки минус продажи (TIP):', list(set(df_stock_tip['tip_product'])
                                                 - set(df_abc_tip_sum['tip_product'])) )

In [ ]:
# df_stock_tip добавляем АБС
df_stock_tip = pd.merge(df_stock_tip, df_abc_tip_sum,
                        on=['tip_product', 'tip_product'], how='left')

In [ ]:
printer(" ")
printer('Остатки_гост после -- df_stock_tip.shape:', df_stock_tip.shape )
printer('Остатки_гост после -- len(df_stock_tip[tip].unique()):',
           len(df_stock_tip['tip_product'].unique()) )

# Подготовка к объединению

In [ ]:
# проверка размеров
print(df_order_tip.shape)
print(len(df_order_tip['tip_product'].unique()))

# проверка размеров
print(df_stock_tip.shape)
print(len(df_stock_tip['tip_product'].unique()))

In [ ]:
printer(" ")
printer('Столбцы из остатков переносим в отгрузки (df_order_tip[columns] = df_stock_tip[columns)')
printer('Проверяем разницу в позициях по TIP')
printer('Продажи есть, остатков нет:',
          list(set(list(df_order_tip['tip_product']))
               - set(list(df_stock_tip['tip_product'])))
          )
printer('Остатки есть, продаж нет:', 
           list(set(list(df_stock_tip['tip_product']))
                - set(list(df_order_tip['tip_product'])))
          )

In [ ]:
d1 = df_stock_tip[df_stock_tip['tip_product'].duplicated(keep='first')]
d2 = df_order_tip[df_order_tip['tip_product'].duplicated(keep='first')]

In [ ]:
printer('Проверка дубликатов df_stock_tip:', len(d1)  )
printer('Проверка дубликатов df_order_tip:', len(d2)  )
printer(" ")

In [ ]:
# проверяем разницу в периоде, получаем все столбцы кроме gost и номеров недель
# остатки минус отгрузки
lst_st_minus_or = list(set(df_stock_tip.columns.values) 
                       - set(df_order_tip.columns.values))
# отгрузки минус заказы
lst_or_minus_st = list(set(df_order_tip.columns.values)
                       - set(df_stock_tip.columns.values))

In [ ]:
# делаем сортировку по ГОСТ для остатков и продаж
df_stock_tip = (df_stock_tip
                      .sort_values(['tip_product'], ascending=(False))
                      .reset_index(drop=True)
                     )

In [ ]:
df_order_tip = (df_order_tip
                      .sort_values(['tip_product'], ascending=(False))
                      .reset_index(drop=True)
                     )

In [ ]:
print(df_stock_tip['tip_product'].head(2))
print(df_order_tip['tip_product'].head(2))

In [ ]:
print(df_stock_tip['tip_product'].tail(2))
print(df_order_tip['tip_product'].tail(2))

In [ ]:
# к заказам добавляем данные из остатков, используем список столбцов
# приравниваем по списку столбцов которые есть в остатках, но нет в отгрузках
df_order_tip[lst_st_minus_or] = df_stock_tip[lst_st_minus_or]

In [ ]:
# lst_or_minus_st -'xyz', 'variab'
df_stock_tip[lst_or_minus_st] = df_order_tip[lst_or_minus_st]

In [ ]:
printer('Приравниваем значение по столбцам \
(df_order_tip[columns] = df_stock_tip[columns)')
printer('df_order_tip.shape:', df_order_tip.shape )
printer('df_stock_tip.shape:', df_stock_tip.shape )
printer(" ")

# Формируем приходы понедельно

In [ ]:
printer('Формируем_приходы: df_ssr')
printer('df_stock_tip, df_order_tip, df_receipt_tip')
printer(" ")

In [ ]:
# формируем df_receipt_gost
df_receipt_tip = pd.DataFrame({
    'tip_product': list(df_order_tip['tip_product'])  })

In [ ]:
# объединяем остатки, продажи, приходы: df_stock_gost, df_order_gost, df_receipt_gost
list_df_sor = [df_stock_tip, df_order_tip, df_receipt_tip]
df_ssr = pd.concat(list_df_sor, keys=["stock", "sales", "receipt"]).reset_index()

In [ ]:
# смена названий столбцов столбцов
df_ssr = df_ssr.rename(columns={'level_0': 'status'})
# удаляем старые индексы
df_ssr = df_ssr.drop('level_1', axis=1)
# делаем сортировку по ГОСТ и по status
df_ssr = (df_ssr
                .sort_values(['tip_product', 'status'], ascending=(False, False))
                .reset_index(drop=True)
                     )

In [ ]:
# создаем первый столбец для расчета первого поступления
df_ssr['00-0000'] = None
df_ssr['00-0000'] = df_ssr['00-0000'].fillna(0)

In [ ]:
# задаем нужный порядок столбцов
df_ssr = df_ssr[['status', 'tip_product', '00-0000'] + lst_stock_sort ]

In [ ]:
df_ssr.iloc[0:3, 0:7]

In [ ]:
# # рассчет остатков без учета продаж

# # количество строк
# for lin in range(2, df_ssr.shape[0], 3):
#     # количество стобцов
#     for col in range(2, df_ssr.shape[1]-1):
#         x = df_ssr.iloc[lin-2, col+1] - df_ssr.iloc[lin-2, col]
#         if x > 0:
#             df_ssr.iloc[lin, col] = x

In [ ]:
# рассчет остатков с учетом продаж

# количество строк
for lin in range(2, df_ssr.shape[0], 3):
    # количество стобцов
    for col in range(2, df_ssr.shape[1]-1):
        df_ssr.iloc[lin, col] = (df_ssr.iloc[lin-2, col+1]
                                 - df_ssr.iloc[lin-2, col] + df_ssr.iloc[lin-1, col])        

In [ ]:
# со смещением продаж на +1 - df_ssr.iloc[lin-2, col] + df_ssr.iloc[lin-1, col+1)

In [ ]:
# заполняем последний столбец
df_ssr.iloc[:, -1:None] = df_ssr.iloc[:, -1:None].fillna(0)

In [ ]:
# рассчитываем сумму по каждой строке
df_ssr['sum'] = df_ssr[df_ssr.columns.values[2:]].sum(axis=1)

In [ ]:
# создаем столбец и рассчитываем коэффициент
# формула: liq_index = (sales - receipt) / sales

df_ssr['liq_index'] = None

for l in range(2, df_ssr.shape[0]+1, 3):
    df_ssr.loc[l, 'liq_index'] = (df_ssr.loc[l-1, 'sum'] - df_ssr.loc[l, 'sum']) / df_ssr.loc[l-1, 'sum']

In [ ]:
# заполняем пропуски нулями и создаем группы
df_ssr['liq_index'] = df_ssr['liq_index'].fillna(0)

# помечаем группу
df_ssr['liq_group'] = df_ssr['liq_index'].apply(lambda x:
'N--'   if x <= -1 else
'N-'    if x <= -0.3 else
'Norm'  if x <= -0.1 else
'0'     if x == 0 else
'N+'    if x <= 0.2 else                                                
'N++')

In [ ]:
# отфильтровываем только приходы
df_receipt = df_ssr[df_ssr.status == 'receipt']
len(df_receipt)

In [ ]:
printer(" ")
printer('Отфильтровываем только приходы: df_receipt')
printer('df_ssr.shape:', df_ssr.shape)
printer('df_receipt.shape:', df_receipt.shape)
printer('Проверяем равенство: df_ssr')
printer('Остатки: status == stock',   len(df_ssr.query('status == "stock"'))  )
printer('Продажи: status == sales',    len(df_ssr.query('status == "sales"'))  )
printer('Приходы: status == receipt',    len(df_ssr.query('status == "receipt"'))  )

In [ ]:
df_receipt = df_receipt.sort_values(by='tip_product',
                                    ascending = False).reset_index(drop=True)

In [ ]:
# добавляем столбцы в остатки и заказы
# ['liq_index', 'liq_group']  ->  df_stock_tip    df_order_tip
df_stock_tip[['liq_index', 'liq_group']] = df_receipt[['liq_index', 'liq_group']]
df_order_tip[['liq_index', 'liq_group']] = df_receipt[['liq_index', 'liq_group']]

# Объединяем остатки и продажи

In [ ]:
# объединяем остатки и продажи df_stock_tip, df_order_tip
list_df_so = [df_stock_tip, df_order_tip]
df_stock_and_sales = pd.concat(list_df_so, keys=["stock", "sales"]).reset_index()

# смена названий столбцов 
df_stock_and_sales = df_stock_and_sales.rename(columns={'level_0': 'status'})
# удаляем старые индексы
df_stock_and_sales = df_stock_and_sales.drop('level_1', axis=1)
# делаем сортировку по ГОСТ и по status
df_stock_and_sales = (df_stock_and_sales
                      .sort_values(['tip_product', 'status'], ascending=(False, False))
                      .reset_index(drop=True)
                     )

In [ ]:
printer(" ")
printer('Объединяем_остатки_и_продажи')
printer('df_stock_and_sales = concat(df_stock_tip, df_order_tip)')
printer(" ")
printer('Делаем сортировку по TIP и по status')
printer(" ")
printer('Проверяем перед выводом графиков df_stock_and_sales')
printer('Остатки: status == stock',   len(df_stock_and_sales.query('status == "stock"'))  )
printer('Продажи: status == sales',    len(df_stock_and_sales.query('status == "sales"'))  )
printer('df_stock_and_sales[tip_product].head(4)', df_stock_and_sales['tip_product'].head(4))
printer('df_stock_and_sales[tip_product].tail(4)', df_stock_and_sales['tip_product'].tail(4))
printer(" ")

# Формируем_столбцы_для_вывода

In [ ]:
# создаем список столбцов для вывода

lst_columns_for_charts = [
    'status', 'tip_product', 'code_1c', 'orders_sum_tip',
    'abc_tip',  'liq_group', 'last_stock',
    'order_freq',
    'stock_sales2', 'stock_purch2', 'how_long_to_sale', 'true_percent',
    'true_receipt', 'brand',
# 'liq_index',
    
#'variab', 'xyz',
    
#'order_psc', 'orders_sum_gost', 'abc_gost', 'abc_marg', 'abc_rent', 'adc_w',
#'true_sales', 'rent_rub',  'turnover',
#'purch_sum', 'purch_mean', 'margin_mean',
#'mean_stock_whole_period', 'price_mean', 'sales_half_year', 'percent',   
#'all_sales_whole_period',  'stock_purch1', 'stock_purch2', 'stock_sales1', 'stock_sales2',

#'mean_st_4', 'order_freq', 
#'mean_sales_whole_period',  'weight_psc_round', 
#'abc_M', 'abc_L', 'abc_H', 'abc_O',
#'margin_sum', 'margin_cums_rub', 'sales_cums_rub', 'margin_perc_rub', 'margin_mean', 
#'sales_perc_rub',  'lmho',
   ]

In [ ]:
# создаем df из df_stock_and_sales, устанавливаем нужный порядок столбцов
# убираем лишние столбцы
df = df_stock_and_sales[lst_columns_for_charts + lst_stock_sort]

In [ ]:
df.loc[:, ['status', 'tip_product', 'code_1c']] = df.loc[:, ['status', 'tip_product', 'code_1c']].astype(str)

In [ ]:
# переименовываем столбцы
df = df.rename(columns={
    'status':         'статус',
    'tip_product':    'ТИП',
    'code_1c':        'Код1С',
    'orders_sum_tip': 'сумма_продаж',
    'order_freq':     'частота',
    'abc_tip':        'абс_тип',
    'liq_group':    'ликвидность',
    'last_stock':   'последний_остаток',
    'stock_sales2': 'склад_в_прод2',
    'stock_purch2': 'склад_в_закуп2',
    'true_percent': 'нет_продаж',
    'true_receipt': 'приход',
    'turnover':'оборачиваемость',
                        })

In [ ]:
# убираем inf, используем numpy
df.replace([np.inf , -np.inf ], 0 , inplace= True )
# df = df.replace ( r'^\s\*$' , np.nan , regex= True )

df = df.where(df.notnull(), None)

# заполняем нулями
df = df.fillna(0)

# Добавление данных на страницу

In [ ]:
# время для формирования имени файла
time_start = DT.datetime.now(DT.timezone.utc).astimezone()
time_format = "_%Y-%m-%d_%H-%M"
time = str(f"{time_start:{time_format}}")

# from xlsxwriter.workbook import Workbook
workbook = Workbook("out/illiquid"+time+".xlsx")
# формирование страницы
worksheet = workbook.add_worksheet('ABC_charts')

# формирование заголовков
headings = df.columns.values           #lst_columns_for_charts + lst_stock_sort

                    #строка, столбец, строка, столбец
worksheet.autofilter(0, 1, 0, len(df.columns))      # добавляем автофильтр
worksheet.freeze_panes(1, 1)                        # закрепляем первую строку и первый столбец

worksheet.set_column('A:A', 137)        # ширина первого столбца
worksheet.write_row("B1", headings)     # добавляем заголовки

worksheet.set_column(1, len(df.columns), 2)     # устанавливаем ширину столбцов 2мм
worksheet.set_column(2, len(df.columns), 8)     # устанавливаем ширину столбцов 6мм
worksheet.set_column(3, len(df.columns), 6)     # устанавливаем ширину столбцов 6мм

# создаем список координат (А2, А3...) начинаются с А2
# поэтому первое значение 0+2
list_add_data = []

for i in range(0, len(df)):
    list_add_data.append('B' + str(i+2))
    
# вставляем данные на страницу (А2, А3...)
for i, ii in zip(range(0, len(df)), list_add_data):
    
    worksheet.set_row(i+1, 95)            # задаем высоту строки
    worksheet.write_row(ii, df.loc[i])    # вставляем данные

# проверить данные 
# workbook.close()

# Формирование категорий и значений

In [ ]:
# объявляем переменые column_chart0 и line_chart1 
list_name_column = []  
list_name_chart = []   

for i in range(0, int(len(df)/2)):
    list_name_column.append('column_chart'+str(i))  # столбчатые диаграммы
    list_name_chart.append('line_chart'+str(i))     # линейные диаграммы


# создаем списки и добавляем словари со значением
# "categories": "=Sheet1!$D$1:$I$1"
# "values": ['Sheet1', i, 3, i, 8],    
    
list_column = []
list_chart = []

list_column_dic = []
list_line_dic = []

# 1,3,5   2,4,6
# i координаты для входных данных по нисходящей
# координаты столбцов для входных данных
w = len(lst_columns_for_charts)+1  # длинна списка заголовков == первому столбцу с неделями
for i, ii in zip(range(1, len(df)+1, 2), range(2, len(df)+1, 2)):
                          # координаты для категорий               # {"categories": "=Sheet1!$N$1:$ER$1",
                                          # страница, строка, столбец, строка, столбец
    
    list_column_dic.append({"categories": ['ABC_charts', 0, w, 0, len(df.columns)],
                            "values":     ['ABC_charts', i, w, i, len(df.columns)],})
                                        # страница, строка, столбец, строка, столбец
    list_line_dic.append({"categories": ['ABC_charts', 0, w, 0, len(df.columns)],
                            "values":   ['ABC_charts', ii, w, ii, len(df.columns)],})


for i_column, i_column_dic in zip(list_name_column, list_column_dic):
    i_column = workbook.add_chart({"type": "line"})    # объявляем переменную
    i_column.add_series(i_column_dic)                  # добавляем name, categ-s, values
    list_column.append(i_column)                       # добавляем в новый список

for i_line, i_line_dic in zip(list_name_chart, list_line_dic):
    i_line = workbook.add_chart({"type": "column"})
    i_line.add_series(i_line_dic)
    list_chart.append(i_line)
    
# в этот список добавляем переменные для объединения
list_for_combine = []                         

for i_column, i_chart, i  in zip(list_column, list_chart, range(0, len(df), 2)):
    i_column.combine(i_chart)                            # объединение столбцы + линии
    i_column.set_legend({"position": "none"})            # удаление легенды
    i_column.set_title({"name": df.loc[i, 'ТИП']})      # заголовок .set_title({"name": df.loc[i, 'gost']}) 
    #i_column.set_y_axis({"name": df.loc[i, 'code_1c']})  # подпись по оси Y
    list_for_combine.append(i_column)

In [ ]:
#### Координаты_для_вставки_графиков_KK2_KK4

In [ ]:
# координаты для вставки графиков K2, K4, K6
# CF прописывать в ручную

# формируем список A->AB->WW (len 702 примерно 14 лет)
import string
list_alphabet_short = list(string.ascii_uppercase)
list_alphabet_long = list(string.ascii_uppercase)
for i in list_alphabet_short:
    for ii in list_alphabet_short:
        list_alphabet_long.append(i+ii)

# координаты для вставки графиков K2, K4, K6
list_k = []

# из длинного списка алфавита получаем нужные буквы начала вставки графиков 
# 'CF'+'2' -> start_index+2
start_index = 'A' #list_alphabet_long[len(df.columns)+1]

for index in range(0, len(df), 2):
    list_k.append(start_index+str(index+2))

## на заданные координаты вставляем графики

In [ ]:
# на заданные координаты вставляем графики
for index_k, i_for_combine in zip(list_k, list_for_combine):
    worksheet.insert_chart(index_k, i_for_combine, {'x_scale': 2, 'y_scale': 0.8})

In [ ]:
# убираем lst
df_no_order['code_1c'] = df_no_order['code_1c'].astype(str)

# добавление второй страницы
worksheet = workbook.add_worksheet('No_sales')
worksheet.freeze_panes(1, 0)                      # закрепляем первую строку и 0 столбец
# worksheet.write(0, 1, "abcd") ### 0 строка, 1 столбец
# вставляем данные на страницу где i номер строки
df_no_order = df_no_order.reset_index(drop=True)
for i in range(0, len(df_no_order)):
    if i == 0:
        worksheet.write_row(i, 0, df_no_order.columns.values)
        worksheet.write_row(i+1, 0, df_no_order.loc[i])
    else:
        worksheet.write_row(i+1, 0, df_no_order.loc[i])    # вставляем данные

In [ ]:
# убираем lst
df_no_stock['code_1c'] = df_no_stock['code_1c'].astype(str)

# добавление второй страницы
worksheet = workbook.add_worksheet('No_stock')
worksheet.freeze_panes(1, 0)                      # закрепляем первую строку и 0 столбец
# worksheet.write(0, 1, "abcd") ### 0 строка, 1 столбец
# вставляем данные на страницу где i номер строки
df_no_stock = df_no_stock.reset_index(drop=True)
for i in range(0, len(df_no_stock)):
    if i == 0:
        worksheet.write_row(i, 0, df_no_stock.columns.values)
        worksheet.write_row(i+1, 0, df_no_stock.loc[i])
    else:
        worksheet.write_row(i+1, 0, df_no_stock.loc[i])    # вставляем данные

In [ ]:
# добавление третьей страницы
worksheet = workbook.add_worksheet('Period')
worksheet.write(1, 1, "Дата первой отгрузки в периоде: "+str(sales_first_date)) ### 0 строка, 1 столбец
worksheet.write(2, 1, "Дата последней отгрузки в периоде: "+str(sales_last_date))

worksheet.write(3, 1, "Продажи, первая неделя в периоде: "
                +str(datetime.strptime(str(sales_first_date).replace(' 00:00:00', ''), '%Y-%m-%d').strftime('%V-%Y'))) 
worksheet.write(4, 1, "Продажи, последняя неделя в периоде: "
                +str(datetime.strptime(str(sales_last_date).replace(' 00:00:00', ''), '%Y-%m-%d').strftime('%V-%Y')))

worksheet.write(6, 1, "Остатки, первая неделя в периоде: "+str(stock_first_week)) 
worksheet.write(7, 1, "Остатки, последняя неделя в периоде: "+str(stock_last_week));

## добавление df_ssr

In [ ]:
# убираем inf, используем numpy
df_ssr.replace([np.inf , -np.inf ], 0 , inplace= True )
# df = df.replace ( r'^\s\*$' , np.nan , regex= True )
df_ssr = df_ssr.where(df_ssr.notnull(), None)
# заполняем нулями
df_ssr = df_ssr.fillna(0)

# добавление df_ssr
worksheet = workbook.add_worksheet('df_ssr')
                    #строка, столбец, строка, столбец
worksheet.autofilter(0, 0, 0, len(df_ssr.columns))      # добавляем автофильтр
worksheet.freeze_panes(1, 2)                      # закрепляем первую строку и второй столбец
# worksheet.write(0, 1, "abcd") ### 0 строка, 1 столбец
# вставляем данные на страницу где i номер строки
df_ssr = df_ssr.reset_index(drop=True)
for i in range(0, len(df_ssr)):
    if i == 0:
        worksheet.write_row(i, 0, df_ssr.columns.values)
        worksheet.write_row(i+1, 0, df_ssr.loc[i])
    else:
        worksheet.write_row(i+1, 0, df_ssr.loc[i])    # вставляем данные

## добавляем журнал

In [ ]:
# добавление страницы
worksheet = workbook.add_worksheet('Журнал')
# ширина первого столбца
worksheet.set_column('A:A', 40)        

# записываем данные
for row_num, row_data in enumerate(printer.cg):
    type_data = type(row_data)
    if isinstance(row_data, (str, int, float)):
        worksheet.write(row_num, 0, str(row_data))
    else:
        try:
            # преобразование в строки
            cleaned_data = [str(item) if item is not None else "" for item in row_data]
            worksheet.write_row(row_num, 0, cleaned_data)
        except Exception as e:
            print(f"Ошибка в строке {row_num}: {e}")
            worksheet.write_row(row_num, 0, f"Ошибка в строке {row_num}: {e}")
            continue

In [ ]:
workbook.close()

# Время выполнения проекта

In [ ]:
# время завершени выполнения проекта
time_project_finish = DT.datetime.now(DT.timezone.utc).astimezone()
print('Время выполнения проекта:', time_project_finish - time_project_start )

In [ ]:
# отчет по группам АБС и неликвиду
df_receipt_stat_liq = (df_receipt.groupby(['liq_group'])
                   .agg({'tip_product': 'count'})
                   .reset_index()
                   .sort_values(by='liq_group', ascending=True))

df_stat_abc = (df_stock_and_sales.groupby(['abc_tip'])
                   .agg({'tip_product': 'count'})
                   .reset_index()
                   .sort_values(by='abc_tip', ascending=True))

In [ ]:
df_receipt_stat_liq

In [ ]:
df_stat_abc